### 필요한 모듈 가져오기

In [17]:
import pandas as pd
import numpy as np
import json
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
import joblib

## TODO: 필요한 라이브러리 추가


### seed 값 고정

In [18]:
# 시드값이 고정이 되어 있지 않을 시 시드 값을 고정하여 모델을 학습 후 평가
## TODO: torch, tensorflow 등 사용 시에도 동일한 SEED 값 설정

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

### 데이터셋 불러오기

In [19]:
df = pd.read_csv('./train.csv')

In [20]:
df.head()

,src_ip,src_port,dst_ip,dst_port,protocol,num_packets,time_sequence,length_sequence,direction_sequence,label
0,150.242.169.98,51776,35.244.247.133,443,TCP,25,"[1625572258.480587, 1625572258.480702, 1625572...","[1470, 1470, 610, 40, 104, 210, 390, 454, 620,...","[-1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, ...",38
1,172.16.30.110,49294,93.158.134.119,443,TCP,107,"[1607051927.782215, 1607051927.782324, 1607051...","[1450, 1450, 1450, 221, 52, 52, 52, 52, 132, 1...","[-1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -...",40
2,192.168.50.107,55750,38.121.72.166,443,TCP,213,"[1611731487.806609, 1611731487.806611, 1611731...","[1500, 636, 1151, 52, 52, 116, 52, 98, 95, 87,...","[-1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1,...",57
3,150.242.169.99,50485,59.108.138.228,443,TCP,12,"[1612662987.601396, 1612662987.601527, 1612662...","[1500, 1500, 315, 40, 120, 311, 311, 102, 40, ...","[-1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1]",29
4,150.242.169.100,61777,109.244.244.164,443,TCP,18,"[1621999755.048606, 1621999755.048879, 1621999...","[1472, 1472, 1272, 1095, 40, 40, 40, 40, 120, ...","[-1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1,...",70


### 전처리

In [21]:
# JSON 형식의 문자열을 리스트로 변환

df['length_sequence'] = df['length_sequence'].apply(json.loads)
df['direction_sequence'] = df['direction_sequence'].apply(json.loads)
df['time_sequence'] = df['time_sequence'].apply(json.loads)

In [22]:
# 입력 데이터에 5튜플에 대한 정보를 추가X

def preprocess(df):
	## TODO: 필요한 전처리 작업 수행

	inputs = []
	for _, row in df.iterrows():
		seq = row['length_sequence']
		if len(seq) >= 20:
			inputs.append(seq[:20]) 
		else:
			inputs.append(seq + [3000] * (20 - len(seq)))
		
	return inputs

In [23]:
# 전처리한 데이터를 열에 추가

df['input'] = preprocess(df)

### 학습, 검증 데이터 나누기

In [24]:
# 학습과 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(df['input'], df['label'], test_size=0.2, random_state=SEED)

# 모델 입력을 위해 리스트 형태로 변환
X_train = X_train.tolist()
y_train = y_train.tolist()

X_valid = X_valid.tolist()
y_valid = y_valid.tolist()

### 모델 학습

In [25]:
model = DecisionTreeClassifier(random_state=SEED)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

### 모델 검증

In [26]:
pred = model.predict(X_valid)
print(f"Accuracy: {accuracy_score(y_valid, pred):.4f}")
print(f"F1 Score: {f1_score(y_valid, pred, average='macro'):.4f}")

Accuracy: 0.8032
F1 Score: 0.8009


### 제출 파일 생성

In [27]:
test_df = pd.read_csv('./testcase.csv')

In [28]:
test_df.head()

,src_ip,src_port,dst_ip,dst_port,protocol,num_packets,time_sequence,length_sequence,direction_sequence
0,192.168.50.244,56819,192.0.77.32,443,TCP,27,"[1617178697.800111, 1617178697.800114, 1617178...","[1500, 248, 1500, 1500, 40, 40, 1040, 40, 160,...","[-1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1,..."
1,150.242.169.98,61627,99.86.202.24,443,TCP,21,"[1615726680.921434, 1615726680.921536, 1615726...","[1280, 1280, 1280, 1280, 1280, 772, 772, 52, 5...","[-1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1,..."
2,150.242.169.100,62098,182.254.59.150,443,TCP,24,"[1620715845.65485, 1620715845.654968, 16207158...","[1500, 1500, 1216, 1268, 41, 40, 120, 600, 438...","[-1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -..."
3,150.242.169.99,61645,184.25.254.123,443,TCP,49,"[1618907087.044121, 1618907087.044251, 1618907...","[1500, 1500, 1252, 1500, 52, 52, 52, 1349, 52,...","[-1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1..."
4,192.168.50.213,47564,59.111.181.35,443,TCP,64,"[1610526084.314479, 1610526084.314679, 1610526...","[1500, 1500, 781, 52, 52, 52, 132, 1500, 1500,...","[-1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1,..."


In [29]:
test_df['length_sequence'] = test_df['length_sequence'].apply(json.loads)
test_df['direction_sequence'] = test_df['direction_sequence'].apply(json.loads)
test_df['time_sequence'] = test_df['time_sequence'].apply(json.loads)

In [30]:
# 테스트 데이터 전처리 수행

test_df['input'] = preprocess(test_df)
X_test = test_df['input'].tolist()

In [31]:
# 모델 예측

pred_test = model.predict(X_test)

In [32]:
# 결과물 저장
## TODO: 팀 번호에 맞게 수정해서 저장

TEAM = 50

###########################################################
# 아래 코드는 수정 X                                       #
# torch 또는 tensorflow로 모델 학습 시 모델 저장 함수만 수정 #
###########################################################
submit_df = pd.DataFrame({'label': pred_test})
submit_df.to_csv(f'./res/{TEAM}_submission.csv', index=False)

## TODO: torch 또는 tensorflow로 모델 학습 시 모델 저장 함수만 수정
joblib.dump(model, f'./res/{TEAM}_model.pkl')
###########################################################
# 위 코드는 수정 X                                         #
# torch 또는 tensorflow로 모델 학습 시 모델 저장 함수만 수정 #
###########################################################

['./res/50_model.pkl']